In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from datasets import load_metric, Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)
from einops import rearrange
from helper import VizHelper
import matplotlib.pyplot as plt

from custom_bert import BertForSequenceClassification

comet_ml is installed but `COMET_API_KEY` is not set.


In [3]:
max_seq_length = 128

In [4]:
SOC_DIR = "./contextualizing-hate-speech-models-with-explanations/"

In [5]:
import sys
if SOC_DIR not in sys.path:
    sys.path.append(SOC_DIR)

# AMI18

In [6]:
model_name = "./bert-base-cased_ami18/"
tokenizer_name = "bert-base-cased"

In [7]:
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
def preprocess_text(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=max_seq_length)

In [8]:
train = pd.read_csv("data/miso_train.tsv", sep="\t")
validation = pd.read_csv("data/miso_dev.tsv", sep="\t")
test = pd.read_csv("data/miso_test.tsv", sep="\t")

raw_datasets = DatasetDict(
    train=Dataset.from_pandas(train),
    validation=Dataset.from_pandas(validation),
    test=Dataset.from_pandas(test)
)
raw_datasets = raw_datasets.rename_column("misogynous", "label")
proc_datasets = raw_datasets.map(preprocess_text, batched=True, remove_columns=raw_datasets["train"].features)
proc_datasets.set_format("pt")

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [9]:
model = AutoModelForSequenceClassification.from_pretrained(model_name).eval()
effective_model = BertForSequenceClassification.from_pretrained(model_name).eval()

*** Calling custom BertForSequenceClassification ***


In [15]:
exp = VizHelper(model, tokenizer, raw_datasets["test"], proc_datasets["test"])

## Attention

In [ ]:
exp.show_attention(idx=21, head=3, layer=10)

In [ ]:
exp.show_effective_attention(idx=21, head=3, layer=10)

In [ ]:
idx, head, layer = 21, 7, 2
fig = exp.compare_attentions(idx, head, layer)
fig.savefig(f"plots/comp_attentions_{idx}_{head}_{layer}.png", bbox_inches='tight')

In [ ]:
idx, head, layer = 21, 1, 9
fig = exp.compare_attentions(idx, head, layer)
fig.savefig(f"plots/comp_attentions_{idx}_{head}_{layer}.png", bbox_inches='tight')

In [ ]:
idx, head, layer = 21, 1, 2
fig = exp.compare_attentions(idx, head, layer, effective_model=effective_model, fontsize=18)
fig.savefig(f"plots/comp_attentions_{idx}_{head}_{layer}.png", bbox_inches='tight')

## Classification

In [ ]:
exp.classify(21)

## Gradients

In [ ]:
exp.get_gradient(21)

## SHAP

In [ ]:
shap = exp.get_kernel_shap(21)

## Comparison Table

In [22]:
table = exp.compute_table("You are a smart woman")

Kernel Shap attribution: 100%|██████████| 200/200 [00:03<00:00, 56.46it/s]


In [23]:
table

tokens,[CLS],You,are,a,smart,woman,[SEP]
G,-0.068182,0.045455,0.136364,-0.068182,-0.477273,0.090909,-0.113636
GxI,0.019544,-0.114596,0.288855,0.059800,-0.243387,-0.141043,-0.132776
IntegratedGradients,-0.254826,0.085741,-0.198310,-0.033707,-0.171537,-0.226774,-0.029106
KernelSHAP,-0.003082,-0.005091,-0.032776,0.037113,0.052062,0.358089,0.071247
SOC,0.018453,0.046849,-0.105045,-0.028419,0.069639,0.588796,0.142798


In [25]:
table.to_excel("sample1.xlsx")

In [67]:
ig = exp.get_integrated_gradients(21)

In [68]:
ig

tensor([-0.0248, -0.0417,  0.0793, -0.0551, -0.0767,  0.0380, -0.0477, -0.1268,
        -0.0331, -0.1121,  0.0308, -0.0457, -0.0316, -0.0350, -0.0862, -0.0903,
        -0.0302,  0.0023, -0.0125], dtype=torch.float64,
       grad_fn=<DivBackward0>)

In [44]:
exp.get_soc(21)

> /home/dauin_user/gattanasio/benchmarking-xai-private/contextualizing-hate-speech-models-with-explanations/hiex/soc_api.py(91)word_level_explanation_bert()
     89 
     90         breakpoint()
---> 91         self.output_file.write(s + '\n')
     92 
     93     def hierarchical_explanation_bert(self, input_ids, input_mask, segment_ids, label=None):



ipdb>  inp


array([  101,   178,  5529,  1139,  4736,  2785,   188,  5108,  1377,
       14908, 20239,   188,  8752,  1377,  7959,  7979,  3919,  2053,
         102,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0]

ipdb>  s


--Return--
None
> /home/dauin_user/gattanasio/benchmarking-xai-private/contextualizing-hate-speech-models-with-explanations/hiex/soc_api.py(91)word_level_explanation_bert()
     89 
     90         breakpoint()
---> 91         self.output_file.write(s + '\n')
     92 
     93     def hierarchical_explanation_bert(self, input_ids, input_mask, segment_ids, label=None):



...

'[CLS] i miss my stupid pretty stank dumb whore skank trick bitch ass friends [SEP]'


ipdb>  len(scores)


17


ipdb>  exit


BdbQuit: 

In [21]:
exp.model.device

device(type='cpu')

In [17]:
item = exp._get_item(21)

In [18]:
item.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])